In [33]:
import pandas as pd 

### 首先从labels 文件中提取信息

In [34]:
def contain_apsect(target):
    for i in target:
        if i != '0':
            return True
    return False

In [38]:
def get_info(data_path):
    labels=pd.read_csv(data_path,header=None)
    # labels.info
    label_list = labels.values.tolist()
    # print(len(label_list))   
    # 这个是正常顺序的
    sentences=[]
    targets=[]
    polarity=[]
    opinions=[]

    for i in range(0,len(label_list),4):
        sentence=[]
        target =[]
        target_polarity =[]
        opinion =[]
        for word in label_list[i]:
            if not isinstance(word,str):
                break
            word = word.lower()
            sentence.append(word)
            
        for index in range(len(sentence)):
            opinion.append(label_list[i+1][index])
            target.append(label_list[i+2][index])
            target_polarity.append(label_list[i+3][index])
        if not contain_apsect(target=target):
            # print(1)
            continue
        sentences.append(sentence)
        targets.append(target)
        polarity.append(target_polarity)
        opinions.append(opinion)
    
    return sentences, targets, polarity, opinions

In [84]:
sentences=[]
targets=[]
polarity=[]
opinions=[]
for i in range(4):
    data_path = '/home/wangshuo/codes/labels-extraction/labels/labels{}.CSV'.format(i+1)
    s, t, p, o=get_info(data_path)
    sentences.extend(s)
    targets.extend(t)
    polarity.extend(p)
    opinions.extend(o)
if len(sentences) != len(targets) or len(sentences) != len(polarity) or len(sentences) != len(opinions):
    print("wrong")
else:
    print("data-set number is {} ".format(len(sentences)))


/tmp/ipykernel_10208/3395467893.py:2: DtypeWarning: Columns (43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,229,230,231,232,233,234,235,236,237,238,240,241,242,243,244,245,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,

data-set number is 1507 


In [ ]:
print(sentences)

In [80]:
max =-1 
for sen in sentences:
    if len(sen)>max:
        max = len(sen)
print(max)

146


### 把这几个二维数组变成一维数组
其中截断到前50

In [85]:

def two2one(array):
    new_array = []

    for list in array:
        list = list[0:50]
        new_array.append(' '.join(list))
    return new_array

In [86]:
sentences = two2one(sentences)
targets = two2one(targets)
polarity = two2one(polarity)
opinions = two2one(opinions)

In [ ]:
print(sentences)

### 生成训练集、测试集、验证集 10:3:3


In [45]:
## 随机生成index
import random
seed_test = 1111
seed_val = 2222

data_size = len(sentences)
index_list = range(data_size)

random.seed(seed_test)
test_size = int(data_size*0.3)
index_test = random.sample(index_list,test_size)

random.seed(seed_val)
val_size = int(data_size*0.3)
index_val = random.sample(index_list,val_size)

In [47]:
print(len(index_val))
print(len(index_test))
print(index_val[0:10])
print(index_test[0:10])

452
452
[437, 570, 139, 269, 1486, 528, 49, 467, 163, 168]
[445, 404, 705, 1315, 290, 745, 915, 171, 1491, 1032]


In [51]:
def get_data(array,indexs):
    new_array = []
    for index in indexs:
        new_array.append(array[index])
    return new_array

In [52]:
def split_data(sentences, targets, polarity, opinions, indexs):
    return get_data(sentences,indexs), get_data(targets,indexs),  get_data(polarity,indexs), get_data(opinions,indexs)
    


### 选取划分index中item

In [88]:
test_sentences, test_targets,test_polar,test_opi = split_data(sentences, targets, polarity, opinions, index_test)
val_sentences, val_targets,val_polar,val_opi = split_data(sentences, targets, polarity, opinions, index_val)


### 划分后的写入到文件中

In [55]:
def write_path(path,array):
    with open(path,'w') as f :
        f.write('\n'.join(array))
    f.close()

In [ ]:
print(sentences)

In [56]:
import os 
def write_file(sentences, targets, polarity, opinions,data_path):
    sentences_path = os.path.join(data_path,'sentence.txt')
    targets_path = os.path.join(data_path,'target.txt')
    polarity_path = os.path.join(data_path,'target_polarity.txt')
    opinions_path = os.path.join(data_path,'opinion.txt')
    write_path(sentences_path,sentences)
    write_path(targets_path,targets)
    write_path(polarity_path,polarity)
    write_path(opinions_path,opinions)

In [89]:
train_path = '/home/wangshuo/codes/labels-extraction/cell/train'
test_path = '/home/wangshuo/codes/labels-extraction/cell/test'
dev_path = '/home/wangshuo/codes/labels-extraction/cell/dev'
write_file(sentences, targets, polarity, opinions,train_path)
write_file(test_sentences, test_targets,test_polar,test_opi,test_path)
write_file(val_sentences, val_targets,val_polar,val_opi,dev_path)
